In [30]:
import numpy as np
import pandas as pd
from xbbg import blp
import statsmodels.api as sm
# vectorbt is imported but not required for the analytical backtest path
import vectorbt as vbt

# =========================================================
# Config
# =========================================================
start = '2007-06-01'   # UUP/BIL exist by ~mid-2007
end = None             # None -> today
use_upro = False       # True: use UPRO (3x) equity fallback; False: SPY
fees = 0.001           # 10 bps per trade (round-trip ~20 bps); used if you later switch to order-based BT
slippage = 0.0
initial_capital = 1_000_000

def_assets = [
    'TLT US Equity',
    'GLD US Equity',
    'DBC US Equity',
    'UUP US Equity',
]
equity_ticker = 'UPRO US Equity' if use_upro else 'SPY US Equity'
cash_ticker = 'BIL US Equity'
all_tickers = def_assets + [equity_ticker, cash_ticker]

lookbacks = [1, 3, 6, 12]  # months
alloc_map = {1: 0.40, 2: 0.30, 3: 0.20, 4: 0.10}

# =========================================================
# Helpers to handle MultiIndex layout differences
# =========================================================
def extract_field(df, field_name):
    """
    Handle BDH outputs where columns may be:
      - MultiIndex (field, ticker)
      - MultiIndex (ticker, field)
      - Single-level (already extracted)
    Return DataFrame with columns as tickers for the requested field.
    """
    if df is None or df.empty:
        return pd.DataFrame()
    cols = df.columns
    if isinstance(cols, pd.MultiIndex):
        # Try (field, ticker)
        if field_name in cols.get_level_values(0):
            try:
                return df.xs(field_name, axis=1, level=0)
            except Exception:
                pass
        # Try (ticker, field)
        if field_name in cols.get_level_values(1):
            try:
                return df.swaplevel(0, 1, axis=1).xs(field_name, axis=1, level=0)
            except Exception:
                pass
        return pd.DataFrame(index=df.index)
    # Single-level columns; assume already the field we asked for
    return df.copy()

# =========================================================
# Robust Bloomberg fetch (TRI if possible, else PX_LAST)
# handles MultiIndex either orientation and ensures monthly resample
# =========================================================
def fetch_tri(tickers, start, end):
    if end is None:
        end = pd.Timestamp.today().normalize()
    if isinstance(start, str):
        start = pd.Timestamp(start)
    if isinstance(end, str):
        end = pd.Timestamp(end)

    # TRI first
    tri_raw = blp.bdh(tickers, flds=['TOT_RETURN_INDEX_GROSS_DVDS'],
                      start_date=start, end_date=end)
    tri_df = extract_field(tri_raw, 'TOT_RETURN_INDEX_GROSS_DVDS')

    # PX fallback for missing/all
    need_px = tickers if tri_df.empty else [t for t in tickers if t not in tri_df.columns or tri_df[t].dropna().empty]
    if need_px:
        px_raw = blp.bdh(need_px, flds=['PX_LAST'], start_date=start, end_date=end)
        px_df = extract_field(px_raw, 'PX_LAST')
        if tri_df.empty:
            df = px_df.copy()
        else:
            df = tri_df.join(px_df, how='outer')
    else:
        df = tri_df.copy()

    if df is None or df.empty:
        raise ValueError("No data returned from Bloomberg for requested tickers/dates.")

    # Ensure DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index, errors='raise')

    df = df.sort_index().dropna(axis=1, how='all')
    if df.index.has_duplicates:
        df = df[~df.index.duplicated(keep='last')]

    # Resample to month-end
    df_m = df.resample('M').last()
    return df_m

# =========================================================
# Strategy functions
# =========================================================
def composite_momentum(prices_monthly, lookbacks):
    parts = []
    for k in lookbacks:
        parts.append(prices_monthly / prices_monthly.shift(k) - 1.0)
    return sum(parts) / len(parts)

def build_weights(mom_df, def_assets, cash_ticker, equity_ticker, alloc_map):
    # Rank defensive assets (1 = best)
    ranks = mom_df[def_assets].rank(axis=1, ascending=False, method='first')

    # Base weights by rank
    cols = def_assets + [equity_ticker]
    base_w = pd.DataFrame(0.0, index=mom_df.index, columns=cols)
    for r, w in alloc_map.items():
        for t in def_assets:
            base_w.loc[ranks[t] == r, t] = w

    # Cash filter: if defensive momentum < cash momentum, reassign that slice to equity
    mom_def = mom_df[def_assets]
    cash_mom = mom_df[cash_ticker]
    fail_mask = mom_def.lt(cash_mom, axis=0)

    failed_w = base_w[def_assets].where(fail_mask, 0.0).sum(axis=1)
    adj_def_w = base_w[def_assets].mask(fail_mask, 0.0)
    equity_w = base_w[equity_ticker].copy().add(failed_w, fill_value=0.0)

    weights = pd.concat([adj_def_w, equity_w.rename(equity_ticker)], axis=1)
    s = weights.sum(axis=1).replace(0, np.nan)
    weights = weights.div(s, axis=0).fillna(0.0)
    return weights

# =========================================================
# Pipeline (data, signals, weights)
# =========================================================
prices_m = fetch_tri(all_tickers, start, end)

# Ensure we have all tickers
needed = [*def_assets, cash_ticker, equity_ticker]
missing_needed = [t for t in needed if t not in prices_m.columns]
if missing_needed:
    raise ValueError(f"Missing required tickers in data: {missing_needed}")

# Monthly returns (for diagnostics and analytic path)
ret_m = prices_m.pct_change().dropna()

# Composite momentum and alignment
mom = composite_momentum(prices_m, lookbacks).dropna()
common_idx = ret_m.index.intersection(mom.index)
ret_m = ret_m.loc[common_idx]
mom = mom.loc[common_idx]

# Monthly target weights
weights_m = build_weights(mom, def_assets, cash_ticker, equity_ticker, alloc_map)

# Price matrix aligned to weights for backtest
px_bt = prices_m[weights_m.columns].reindex(weights_m.index).ffill()

# Benchmark preparation (SPY buy & hold)
bench = None
if 'SPY US Equity' in prices_m.columns:
    spy_px = prices_m[['SPY US Equity']].reindex(px_bt.index).ffill()
    spy_ret = spy_px.pct_change().squeeze().dropna()
    spy_equity = (1 + spy_ret).cumprod() * initial_capital

# =========================================================
# Version-agnostic backtest (analytical monthly returns)
# Portfolio monthly return = sum_t weights_t * asset_return_{t}
# =========================================================
ret_m_aligned = px_bt.pct_change().loc[weights_m.index]
port_ret = (weights_m * ret_m_aligned).sum(axis=1).dropna()

equity = (1 + port_ret).cumprod() * initial_capital

class SimplePF:
    def __init__(self, equity, returns):
        self._equity = equity
        self._returns = returns
    def value(self):
        return self._equity
    def returns(self):
        return self._returns
    def stats(self, settings=None):
        df = self._returns
        ann_factor = 12  # monthly data
        T = df.shape[0]
        ann_ret = (1 + df).prod() ** (ann_factor / T) - 1 if T > 0 else np.nan
        vol = df.std() * np.sqrt(ann_factor) if T > 1 else np.nan
        sharpe = ann_ret / vol if vol and vol > 0 else np.nan
        mdd = (self._equity / self._equity.cummax() - 1).min()
        return pd.Series({
            'Total Return [%]': (self._equity.iloc[-1] / self._equity.iloc[0] - 1) * 100,
            'Annual Return [%]': ann_ret * 100,
            'Volatility [%]': vol * 100,
            'Sharpe Ratio': sharpe,
            'Max Drawdown [%]': mdd * 100,
            'Hit Rate (months) [%]': (df > 0).mean() * 100
        })

pf = SimplePF(equity, port_ret)

# Benchmark as SimplePF (if prepared)
if 'spy_ret' in locals() and 'spy_equity' in locals():
    bench = SimplePF(spy_equity, spy_ret)
else:
    bench = None

# =========================
# Print basic performance
# =========================
print("Strategy stats:")
print(pf.stats())

if bench is not None:
    print("\nSPY (buy & hold) stats:")
    print(bench.stats())

# =========================
# Diagnostics and extras
# =========================
# Average next-month return by momentum rank (defensive assets)
defense_ranks = mom[def_assets].rank(axis=1, ascending=False, method='first')
next_def_ret = ret_m[def_assets].shift(-1)
rank_long = (
    defense_ranks.stack().rename('rank')
    .to_frame()
    .join(next_def_ret.stack().rename('next_ret'))
    .dropna()
)
avg_next_by_rank = rank_long.groupby('rank')['next_ret'].mean().sort_index()
print("\nAverage next-month return by momentum rank (defensive assets):")
print(avg_next_by_rank)

# Fama–MacBeth: cross-sectional regression of future returns on scores
scores = mom[def_assets]
future = ret_m[def_assets].shift(-1)
common = scores.index.intersection(future.index)
scores = scores.loc[common]
future = future.loc[common]

betas = []
dates_used = []
for dt in common:
    y = future.loc[dt].dropna()
    x = scores.loc[dt].reindex(y.index)
    if len(y) < 3:
        continue
    X = sm.add_constant(x.values)
    model = sm.OLS(y.values, X).fit()
    betas.append(model.params[1])  # slope on score
    dates_used.append(dt)

betas = pd.Series(betas, index=pd.Index(dates_used, name='date'))
beta_mean = betas.mean()

# Newey–West HAC SE for mean of betas (Bartlett kernel) - robust implementation
lag = 6
T = len(betas)

if T == 0:
    se_mean = np.nan
    t_stat = np.nan
else:
    eps = (betas - beta_mean).to_numpy()
    gamma0 = np.dot(eps, eps) / T
    hac_var = gamma0
    max_L = min(lag, T - 1)
    for L in range(1, max_L + 1):
        w = 1 - L / (lag + 1)
        gammaL = np.dot(eps[L:], eps[:-L]) / T
        hac_var += 2 * w * gammaL
    se_mean = np.sqrt(hac_var / T)
    t_stat = beta_mean / se_mean if se_mean > 0 else np.nan

print("\nFama–MacBeth: slope on composite momentum score")
print(f"Average slope: {beta_mean:.4f}, Newey–West t-stat (lag={lag}): {t_stat:.2f}")

# =========================
# Optional exports / visuals
# =========================
# equity.to_csv('strategy_equity.csv')
# port_ret.to_csv('strategy_monthly_returns.csv')
# weights_m.to_csv('strategy_weights_monthly.csv')
# contrib_by_asset = (weights_m * ret_m_aligned).sum()  # total contribution by asset
# contrib_by_asset.to_csv('strategy_total_contribution_by_asset.csv')

# Example quick plots (uncomment if running interactively):
# equity.plot(title='Strategy Equity'); import matplotlib.pyplot as plt; plt.show()
# (port_ret.rolling(12).mean() * 12).plot(title='Rolling 12M Mean Return (annualized)'); plt.show()
# (port_ret.rolling(12).std() * np.sqrt(12)).plot(title='Rolling 12M Volatility'); plt.show()

Strategy stats:
Total Return [%]         3144.257937
Annual Return [%]          22.349132
Volatility [%]              8.613519
Sharpe Ratio                2.594657
Max Drawdown [%]           -6.915056
Hit Rate (months) [%]      77.294686
dtype: float64

SPY (buy & hold) stats:
Total Return [%]         599.866415
Annual Return [%]         11.957003
Volatility [%]            15.707839
Sharpe Ratio               0.761212
Max Drawdown [%]         -41.707759
Hit Rate (months) [%]     67.475728
dtype: float64

Average next-month return by momentum rank (defensive assets):
rank
1.0    0.007460
2.0    0.001904
3.0    0.002696
4.0   -0.001229
Name: next_ret, dtype: float64

Fama–MacBeth: slope on composite momentum score
Average slope: 0.0763, Newey–West t-stat (lag=6): 2.67
